## Using BeautifulSoup

BeautifulSoup is a library which parses HTML and creates a tree structure of python objects that we can navigate through, extract information from, and edit

In [48]:
#!conda install -y bs4

#### Convert the raw HTML string to a BeautifulSoup object

In [49]:
import requests
from bs4 import BeautifulSoup

### Task 1: Extract the poem titles, url and poems with Beautiful Soup

In [50]:
def search_url(url): 
    import os
    import time
    """
    search_url is a function that accepts as parameter a url string
    and then returns a text file named as text with the web page
    requested. 
    It prints in the excecustion if the code is corectly downloaded.
    It uses request package!
    """                                                                                               
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header)
    time.sleep(3)
    print(f'The status code is: {response.status_code}')
    filename = f"response_files/response.txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        f.write(response.text)
    text = response.text

    return text

In [89]:
def find_songs(class_of,saving,artist):
    """
    Makes a Data Frame with the title, url, and lyrics as columns.
    As parameter you have to include the class of the list of songs on the url.
    Be careful about the number of \n in the title and extract them.
    """
    import time
    
    all_song_titles = soup.find_all(class_= class_of)
    all_song_titles
    titles = []
    for title in all_song_titles:
        titles.append(title.a.text.split('\n')[0].strip()) 
    # Extract all links
    links = []
    songlinks = soup.find_all(class_= class_of)
    for link in songlinks:
        links.append(link.a['href'])
    import pandas as pd
    df = pd.DataFrame({'title':titles,'links':links})
    df['artist'] = artist
    df['song_lyric'] = 'blank'
    for i in range(len(df.links[:])):
        try:
            lyric = BeautifulSoup(requests.get(df.links[i]).text,)
            df['song_lyric'][i] = lyric.find('p',attrs={'class':'song-lyrics'}).text
            time.sleep(3)
        except:
            continue
    if saving == True:
        import os
        for i in range(len(df.song_lyric[:])):
            filename = f"response_files/lyrics/{df.title[i]}.txt"
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename,'w') as f:
                f.write(df.song_lyric[i])
                f.close()
    return df 

In [90]:

text = search_url('https://lyrics.az/bob-dylan/allalbums.html')
soup = BeautifulSoup(text,'html.parser')
df1 = find_songs('mt-3 col-sm-9 col-12 list-group mt-sm-0',True,'bob_dylan')

text = search_url('https://lyrics.az/eminem/allalbums.html')
soup = BeautifulSoup(text,'html.parser')
df2 = find_songs('mt-3 col-sm-9 col-12 list-group mt-sm-0',True,'eminem')

# text = search_url('https://lyrics.az/simon-and-garfunkel/allsongs.html')
# soup = BeautifulSoup(text,'html.parser')
# df3 = find_songs('px-0 mx-0 mb-5 col-12 col-sm-6 table-responsive',True,'simon&granfunkel')


The status code is: 200


In [72]:
df1

,title,links,artist,song_lyric
0,As Time Goes By,https://lyrics.az/bob-dylan/triplicate/as-time...,bob_dylan,You must remember this\nA kiss is still a kiss...
1,I Shall Be Released,https://lyrics.az/bob-dylan/blood-in-my-eye/i-...,bob_dylan,They say every man must need protection\nThey ...
2,All Or Nothing At All,https://lyrics.az/bob-dylan/fallen-angels/all-...,bob_dylan,All or nothing at all\nHalf a love never appea...
3,Bye Bye Johnny,https://lyrics.az/bob-dylan/live-on-air-1986/b...,bob_dylan,Bye and bye\nI'm breathin' a lover's sigh\nWel...
4,Jet Pilot,https://lyrics.az/bob-dylan/the-bootleg-series...,bob_dylan,"Well, she's got jet pilot eyes from her hips o..."
...,...,...,...,...
75,Ballad In Plain D,https://lyrics.az/bob-dylan/another-side-of-bo...,bob_dylan,"I once loved a girl, her skin it was bronze\nW..."
76,Untitled 1 (Baby's Black),https://lyrics.az/bob-dylan/some-other-kinds-o...,bob_dylan,baby's black\nbeen had\nain't bad\nsmokestacke...
77,Ballad Of Hollis Brown,https://lyrics.az/bob-dylan/the-times-they-are...,bob_dylan,Hollis Brown\nHe lived on the outside of town\...
78,Bob Dylan's Blues,https://lyrics.az/bob-dylan/the-freewheelin-bo...,bob_dylan,"Well, the Lone Ranger and Tonto\nThey are ridi..."


In [130]:
df2 

,title,links,artist,song_lyric
0,Alfred (Interlude),https://lyrics.az/eminem/music-to-be-murdered/...,eminem,How do you do? Ladies and gentlemen\nMy name i...
1,Em Calls Paul (Skit) (2018),https://lyrics.az/eminem/kamikaze/em-calls-pau...,eminem,"[Skit: Eminem & GPS]\nPaul, yo, um\nSo I got y..."
2,Revival,https://lyrics.az/eminem/campaign-speech/reviv...,eminem,"[Verse]\nI’ve drowned in high from the ashes, ..."
3,1-833-2GET-REV,https://lyrics.az/eminem/revival/1-833-2get-re...,eminem,"Thank you for your interest in Revival, the nu..."
4,8 Mile: Final Battle,https://lyrics.az/eminem/8-mile-motion-picture...,eminem,"[Rabbit]\nNow, everybody from the 313\nPut you..."
5,Hail Mary,https://lyrics.az/eminem/dj-green-lantern-mixt...,eminem,[Intro: Eminem]\nMakaveli rest in peace\nIrv G...
6,Brain Damage,https://lyrics.az/eminem/eminem-unmastered-seq...,eminem,"""Scalpel""\n""Here""\n""Sponge""\n""Here""\n""Wait he'..."
7,American Psycho,https://lyrics.az/eminem/eminem-vs-dj-whoo-kid...,eminem,"[Verse 1: Slim Shady]\nI'm the devil, if ever ..."
8,3 Verses,https://lyrics.az/eminem/f**ing-crazy/3-verses...,eminem,I'm the illest rapper to hold a cordless\nPatr...
9,Campaign Speech,https://lyrics.az/eminem/for-president/campaig...,eminem,Jumped out of the second floor of a record sto...


# Bag of words

In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [151]:
def vect_fit_transf(sample, df): # sample is the df.song_lyric[:2]
    vectorizer = CountVectorizer(stop_words='english',max_df=0.8)#,ngram_range=(1, 2))
    X = vectorizer.fit_transform(sample)
    X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names(),index = df['artist'])
    y_df = df.artist

    return X_df , y_df

##### How can we remove the most common words?

* Using a list of stop words
* Removing the words that appear in more than X% of documents

In [152]:
X_df , y_df =vect_fit_transf(df1.song_lyric[:],df1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [153]:
X_df.shape, y_df.shape

((80, 3110), (80,))

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [155]:
X_df.head()

,1910,1998,7th,90,abandon,abandoned,abiding,ability,abuse,accept,...,yes,yesterday,yield,yoga,yon,yonder,young,youre,yuh,zy
artist,,,,,,,,,,,,,,,,,,,,,
bob_dylan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bob_dylan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bob_dylan,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
bob_dylan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bob_dylan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
from collections import Counter
c = Counter(X_df)
print(c.most_common(3))

    

[('1910', 1), ('1998', 1), ('7th', 1), ('90', 1)]


In [ ]:
np.square(X_df).sum(axis=1)

Beatles    1.0
Beatles    1.0
Eminem     1.0
Eminem     1.0
dtype: float64

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [162]:
vectorizer = TfidfVectorizer(tokenizer=None)
X = vectorizer.fit_transform(df1.song_lyric[:])
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names(), index=df1['artist'])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [163]:
X_df

,1910,1998,7th,90,abandon,abandoned,abiding,ability,about,above,...,yoga,yon,yonder,you,young,your,youre,yourself,yuh,zy
artist,,,,,,,,,,,,,,,,,,,,,
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.099358,0.000000,0.000000,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.087920,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.027617,0.000000,0.192288,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.144243,0.000000,0.000000,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.038340,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025464,0.000000,...,0.0,0.0,0.000000,0.012098,0.070135,0.000000,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063488,0.000000,...,0.0,0.0,0.000000,0.060330,0.000000,0.000000,0.0,0.0,0.0,0.0
bob_dylan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.070286,...,0.0,0.0,0.000000,0.169742,0.000000,0.472745,0.0,0.0,0.0,0.0


In [164]:
np.square(X_df).sum(axis=1)

artist
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
            ... 
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
bob_dylan    1.0
Length: 80, dtype: float64

In [165]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print(operator.itemgetter(*np.argsort(model.coef_[0]))(vectorizer.get_feature_names())[-20:])
print(operator.itemgetter(*np.argsort(model.coef_[0]))(vectorizer.get_feature_names())[:20])

AttributeError: 'DataFrame' object has no attribute 'itemgetter'